In [1]:
cd ..

/Users/anthonycuturrufo/Documents/Research/Wang_Lab/Repo/KG_Link_Prediction/KG-Link_Prediction


In [2]:
#Code based on https://www.dgl.ai/news/2020/06/09/covid.html

In [23]:
from lib import format_transformer

In [24]:
import pandas as pd
import numpy as np

drkg_file = './data/v1_res/relation/DDires.csv'
df = pd.read_csv(drkg_file)
df.head()

,Drug,Disease,Palliates_Hetionet,Treats_Hetionet,Effect_KEGG,Association_CTD
0,DrugBank:DB00997,DOID:363,0,1,0,1
1,DrugBank:DB00206,DOID:10763,0,1,0,1
2,DrugBank:DB00960,DOID:10763,0,1,0,1
3,DrugBank:DB00665,DOID:10283,0,1,1,1
4,DrugBank:DB00290,DOID:2998,0,1,0,1


In [25]:
triples = format_transformer.transform_kg_csv_to_triplets([drkg_file])

In [26]:
triples[0:5]

[['DrugBank:DB00997', 'Treats_Hetionet', 'DOID:363'],
 ['DrugBank:DB00997', 'Association_CTD', 'DOID:363'],
 ['DrugBank:DB00206', 'Treats_Hetionet', 'DOID:10763'],
 ['DrugBank:DB00206', 'Association_CTD', 'DOID:10763'],
 ['DrugBank:DB00960', 'Treats_Hetionet', 'DOID:10763']]

In [27]:
len(triples)

2685182

In [28]:
format_transformer.partition_dataset(triples, "WANGKG", .9, .05)

In [39]:
from csv import reader
train_fp = "notebooks/train/{name}_train.tsv".format(name="WANGKG")

with open(train_fp, 'r') as read_obj:
    csv_reader = reader(read_obj, delimiter = "\t")
    # Pass reader object to list() to get a list of lists
    list_of_rows = list(csv_reader)
    


In [41]:
list_of_rows[0:10]

[['MESH:C032036', 'Association_CTD', 'DOID:12270'],
 ['MESH:C065250', 'Association_CTD', 'DOID:4535'],
 ['MESH:C044578', 'Association_CTD', 'DOID:2237'],
 ['DrugBank:DB00997', 'Association_CTD', 'PharmGKB:PA153619833'],
 ['DrugBank:DB01452', 'Association_CTD', 'MESH:D012883'],
 ['MESH:C577929', 'Association_CTD', 'MESH:D003921'],
 ['DrugBank:DB00627', 'Association_CTD', 'KEGG:H01723'],
 ['MESH:D011509', 'Association_CTD', 'DOID:1227'],
 ['MESH:C022894', 'Association_CTD', 'PharmGKB:PA444547'],
 ['DrugBank:DB01101', 'Association_CTD', 'PharmGKB:PA444178']]

In [42]:
df_test2 = pd.DataFrame(list_of_rows, columns=["h", "r", "t"])

In [43]:
num_nodes = len(np.unique(df_test2.h.values))

In [44]:
num_nodes

16251

In [29]:
!DGLBACKEND=pytorch dglke_train --dataset WANGKG --data_path ./notebooks/train \
                               --data_files drkg_train.tsv drkg_valid.tsv drkg_test.tsv \
                               --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 \
                               --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 \
                               --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv \
                               --regularization_coef 1.00E-07 --test --num_thread 1 \
                               --gpu 0 1 2 3 4 5 6 7 --num_proc 8 \
                               --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Traceback (most recent call last):
  File "/Users/anthonycuturrufo/opt/anaconda3/bin/dglke_train", line 8, in <module>
    sys.exit(main())
  File "/Users/anthonycuturrufo/opt/anaconda3/lib/python3.8/site-packages/dglke/train.py", line 81, in main
    dataset = get_dataset(args.data_path,
  File "/Users/anthonycuturrufo/opt/anaconda3/lib/python3.8/site-packages/dglke/dataloader/KGDataset.py", line 649, in get_dataset
    dataset = KGDatasetUDDRaw(data_path, data_name, delimiter, files, format, has_edge_importance)
  File "/Users/anthonycuturrufo/opt/anaconda3/lib/python3.8/site-packages/dglke/dataloader/KGDataset.py", line 415, in __init__
    assert os.path.exists(os.path.join(path, f)), \
AssertionError: File drkg_train.tsv not exist in ./notebooks/train
